In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [23]:
import tiktoken
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002").encode, verbose= True)
callback_manager = CallbackManager([token_counter])
Settings.callback_manager = callback_manager 

In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('assets/sleep').load_data()
index = VectorStoreIndex.from_documents(documents, show_progress=True)
index.storage_context.persist(persist_dir='./storage/cache/sleep')
print('persisting to disk')

print(token_counter.total_embedding_token_count)

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/176 [00:00<?, ?it/s]2025-10-05 12:09:17,700 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Embedding Token Usage: 95311


Generating embeddings:  57%|█████▋    | 100/176 [00:02<00:02, 35.59it/s]2025-10-05 12:09:20,187 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Embedding Token Usage: 72462


Generating embeddings: 100%|██████████| 176/176 [00:05<00:00, 33.32it/s]


persisting to disk
167773


In [36]:
from llama_index.core.prompts import PromptTemplate

text_qa_template_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Using both the context information and also using your own knowledge, "
    "answer the question: {query_str}\n"
    "If the context isn't helpful, you can also answer the question on your own.\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [31]:
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How does sleep enhance learning memory?")
print(response)

2025-10-05 12:17:25,980 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Embedding Token Usage: 7


2025-10-05 12:17:27,674 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM Prompt Token Usage: 2018
LLM Completion Token Usage: 131
Sleep enhances learning and memory by playing a crucial role in the consolidation of information acquired during wakefulness. Slow wave sleep, which occurs primarily in the early part of the night, is important for motor learning and the acquisition of detailed information. During slow wave sleep, the brain undergoes processes that strengthen neural connections related to the learned information, leading to improved retention and recall. Additionally, REM sleep, which occurs throughout the night with a higher percentage towards morning, is associated with emotional processing and memory consolidation. Overall, both slow wave sleep and REM sleep contribute to the enhancement of learning and memory by facilitating the storage and organization of newly acquired information in the brain.


In [32]:
query_engine = index.as_query_engine()

In [33]:
from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("what can i do to get better sleep?")
pprint_response(response, show_source=True)
print(response)

2025-10-05 12:17:28,017 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Embedding Token Usage: 9


2025-10-05 12:17:29,237 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM Prompt Token Usage: 2071
LLM Completion Token Usage: 102
Final Response: To get better sleep, you can try going to bed and
waking up at the same time every day to establish a regular sleep
schedule. Avoid consuming a lot of fluids right before bedtime to
prevent disruptions due to needing to use the bathroom during the
night. Engaging in resistance exercise can help increase slow wave
sleep, which is beneficial for overall sleep quality. Additionally, be
mindful of your caffeine intake throughout the day and consider
avoiding naps, especially in the late afternoon or evening, to ensure
better sleep at night.
______________________________________________________________________
Source Node 1/2
Node ID: df51e61d-780e-4cec-8221-cceb630f463c
Similarity: 0.844787653556624
Text: For every hour variation in sleep regardless of whether or not
it was more sleep than one typically got, there was a 17% reduction in
performance on this particular exam type. So this is powerful. This
means tha

In [34]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = VectorIndexRetriever(index = index, similarity_top_k = 4)
query_engine = RetrieverQueryEngine(
    retriever = retriever
)

In [35]:
from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("what can i do to get better sleep?")
pprint_response(response, show_source=True)
print(response)

2025-10-05 12:17:30,048 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Embedding Token Usage: 9


2025-10-05 12:17:32,122 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM Prompt Token Usage: 4036
LLM Completion Token Usage: 127
Final Response: To get better sleep, you can focus on optimizing your
sleep environment and routine. Utilize tools such as maintaining a
consistent sleep schedule, ensuring your bedroom is cool and dark,
avoiding consuming caffeine too late in the day, limiting naps,
engaging in relaxation techniques like NSDR or Reveri sleep hypnosis,
and considering tools like eye masks and earplugs. Additionally, you
can explore methods to regulate your body temperature, such as cooling
or heating your mattress at different times during the night. Regular
exposure to natural light during the day and minimizing exposure to
artificial light at night can also help regulate your circadian rhythm
for better sleep.
______________________________________________________________________
Source Node 1/4
Node ID: df51e61d-780e-4cec-8221-cceb630f463c
Similarity: 0.8448167765724572
Text: For every hour variation in sleep regardless of whether or not
i